# GCN practice code

- import basic library

In [138]:
import torch
from torch_geometric.data import Data
from torch_geometric.utils import from_networkx

import networkx as nx
import numpy as np
from random import randint, expovariate
import matplotlib.pyplot as plt

- Generate the network

In [139]:
S_CPU_MAX = []
S_BW_MAX = []
net = nx.gnm_random_graph(n=20, m=100)

min_cpu_capacity = 1.0e10
max_cpu_capacity = 0.0
for node_id in net.nodes:
    net.nodes[node_id]['CPU'] = randint(50, 100)
    net.nodes[node_id]['LOCATION'] = randint(0, 2)
    if net.nodes[node_id]['CPU'] < min_cpu_capacity:
        min_cpu_capacity = net.nodes[node_id]['CPU']
    if net.nodes[node_id]['CPU'] > max_cpu_capacity:
        max_cpu_capacity = net.nodes[node_id]['CPU']
        
min_bandwidth_capacity = 1.0e10
max_bandwidth_capacity = 0.0
for edge_id in net.edges:
    net.edges[edge_id]['bandwidth'] = randint(50, 100)
    if net.edges[edge_id]['bandwidth'] < min_bandwidth_capacity:
        min_bandwidth_capacity = net.edges[edge_id]['bandwidth']
    if net.edges[edge_id]['bandwidth'] > max_bandwidth_capacity:
        max_bandwidth_capacity = net.edges[edge_id]['bandwidth']
        
for s_node_id, s_node_data in net.nodes(data=True):
    S_CPU_MAX.append(s_node_data['CPU'])
# S_BW_Free
for s_node_id in range(len(net.nodes)):
    total_node_bandwidth = 0.0
    for link_id in net[s_node_id]:
        total_node_bandwidth += net[s_node_id][link_id]['bandwidth']
    S_BW_MAX.append(total_node_bandwidth)

In [140]:
 # S_CPU_Free
s_CPU_remaining = []
s_bandwidth_remaining = []
current_embedding = [0] * len(net.nodes)

for s_node_id, s_node_data in net.nodes(data=True):
    s_CPU_remaining.append(s_node_data['CPU'])
# S_BW_Free
for s_node_id in range(len(net.nodes)):
    total_node_bandwidth = 0.0
    for link_id in net[s_node_id]:
        total_node_bandwidth += net[s_node_id][link_id]['bandwidth']
    s_bandwidth_remaining.append(total_node_bandwidth)

In [141]:
substrate_features = []
substrate_features.append(S_CPU_MAX)
substrate_features.append(S_BW_MAX)
substrate_features.append(s_CPU_remaining)
substrate_features.append(s_bandwidth_remaining)
substrate_features.append(current_embedding)

print(substrate_features)

[[72, 96, 88, 89, 90, 70, 79, 78, 78, 93, 77, 54, 63, 63, 75, 100, 86, 78, 72, 56], [346.0, 771.0, 702.0, 469.0, 1084.0, 660.0, 804.0, 462.0, 759.0, 1065.0, 865.0, 344.0, 691.0, 967.0, 1262.0, 756.0, 864.0, 767.0, 871.0, 811.0], [72, 96, 88, 89, 90, 70, 79, 78, 78, 93, 77, 54, 63, 63, 75, 100, 86, 78, 72, 56], [346.0, 771.0, 702.0, 469.0, 1084.0, 660.0, 804.0, 462.0, 759.0, 1065.0, 865.0, 344.0, 691.0, 967.0, 1262.0, 756.0, 864.0, 767.0, 871.0, 811.0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]


In [142]:
substrate_features = torch.tensor(substrate_features)
print(substrate_features)
print(substrate_features.shape)
substrate_features = torch.transpose(substrate_features, 0, 1)
print(substrate_features)
print(substrate_features.shape)

tensor([[  72.,   96.,   88.,   89.,   90.,   70.,   79.,   78.,   78.,   93.,
           77.,   54.,   63.,   63.,   75.,  100.,   86.,   78.,   72.,   56.],
        [ 346.,  771.,  702.,  469., 1084.,  660.,  804.,  462.,  759., 1065.,
          865.,  344.,  691.,  967., 1262.,  756.,  864.,  767.,  871.,  811.],
        [  72.,   96.,   88.,   89.,   90.,   70.,   79.,   78.,   78.,   93.,
           77.,   54.,   63.,   63.,   75.,  100.,   86.,   78.,   72.,   56.],
        [ 346.,  771.,  702.,  469., 1084.,  660.,  804.,  462.,  759., 1065.,
          865.,  344.,  691.,  967., 1262.,  756.,  864.,  767.,  871.,  811.],
        [   0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
            0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.]])
torch.Size([5, 20])
tensor([[  72.,  346.,   72.,  346.,    0.],
        [  96.,  771.,   96.,  771.,    0.],
        [  88.,  702.,   88.,  702.,    0.],
        [  89.,  469.,   89.,  469.,    0.],
    

In [121]:
substrate_features = torch.reshape(substrate_features, (-1,))
print(substrate_features.shape)
print(substrate_features)

torch.Size([100])
tensor([ 74., 799.,  74., 799.,   0.,  88., 752.,  88., 752.,   0.,  98., 604.,
         98., 604.,   0.,  51., 622.,  51., 622.,   0.,  80., 930.,  80., 930.,
          0.,  50., 989.,  50., 989.,   0.,  74., 771.,  74., 771.,   0.,  63.,
        431.,  63., 431.,   0.,  64., 970.,  64., 970.,   0.,  99., 867.,  99.,
        867.,   0.,  91., 575.,  91., 575.,   0.,  83., 688.,  83., 688.,   0.,
         92., 831.,  92., 831.,   0.,  86., 801.,  86., 801.,   0.,  64., 894.,
         64., 894.,   0.,  96., 526.,  96., 526.,   0.,  74., 600.,  74., 600.,
          0.,  99., 942.,  99., 942.,   0.,  88., 529.,  88., 529.,   0., 100.,
        629., 100., 629.,   0.])


In [122]:
# vnr_cpu = torch.tensor([10])
# vnr_bw = torch.tensor([30])
# pending = torch.tensor([2])
# substrate_features = torch.cat((substrate_features, vnr_cpu, vnr_bw, pending), 0)

In [123]:
# print(substrate_features)

- Using 'from_networkx'
    - transfer the torch_geometric

In [143]:
data = from_networkx(net)

In [144]:
print(data)

Data(CPU=[20], LOCATION=[20], bandwidth=[200], edge_index=[2, 200])


### Graph Convolution Network
- Generate the GCN class

In [149]:
from torch.nn import Linear
from torch_geometric.nn import GCNConv


class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(in_channels=5, out_channels=4)
        self.conv2 = GCNConv(in_channels=4, out_channels=4)
        self.conv3 = GCNConv(in_channels=4, out_channels=1)
        self.classifier = Linear(1, 20)

    def forward(self, x, edge_index):
        h = self.conv1(x, edge_index)
        h = h.tanh()
        h = self.conv2(h, edge_index)
        h = h.tanh()
        h = self.conv3(h, edge_index)
        h = h.tanh()  # Final GNN embedding space.
        
        # Apply a final (linear) classifier.
        out = self.classifier(h)

        return out, h

model = GCN()
print(model)

GCN(
  (conv1): GCNConv(5, 4)
  (conv2): GCNConv(4, 4)
  (conv3): GCNConv(4, 1)
  (classifier): Linear(in_features=1, out_features=20, bias=True)
)


In [154]:
model = GCN()

print(substrate_features.shape, data.edge_index.shape)
print(substrate_features)

out, embedding = model(substrate_features, data.edge_index)
print(f'Embedding shape: {list(embedding.shape)}')

torch.Size([20, 5]) torch.Size([2, 200])
tensor([[  72.,  346.,   72.,  346.,    0.],
        [  96.,  771.,   96.,  771.,    0.],
        [  88.,  702.,   88.,  702.,    0.],
        [  89.,  469.,   89.,  469.,    0.],
        [  90., 1084.,   90., 1084.,    0.],
        [  70.,  660.,   70.,  660.,    0.],
        [  79.,  804.,   79.,  804.,    0.],
        [  78.,  462.,   78.,  462.,    0.],
        [  78.,  759.,   78.,  759.,    0.],
        [  93., 1065.,   93., 1065.,    0.],
        [  77.,  865.,   77.,  865.,    0.],
        [  54.,  344.,   54.,  344.,    0.],
        [  63.,  691.,   63.,  691.,    0.],
        [  63.,  967.,   63.,  967.,    0.],
        [  75., 1262.,   75., 1262.,    0.],
        [ 100.,  756.,  100.,  756.,    0.],
        [  86.,  864.,   86.,  864.,    0.],
        [  78.,  767.,   78.,  767.,    0.],
        [  72.,  871.,   72.,  871.,    0.],
        [  56.,  811.,   56.,  811.,    0.]])
Embedding shape: [20, 1]


In [151]:
print(embedding)
print(embedding.shape)

tensor([[0.8147],
        [0.9048],
        [0.8893],
        [0.8435],
        [0.9488],
        [0.8798],
        [0.9104],
        [0.8439],
        [0.9060],
        [0.9504],
        [0.9157],
        [0.8028],
        [0.9105],
        [0.9169],
        [0.9504],
        [0.9105],
        [0.9205],
        [0.9106],
        [0.9319],
        [0.9183]], grad_fn=<TanhBackward>)
torch.Size([20, 1])


In [92]:
print(out.shape)
print(out)

torch.Size([20, 20])
tensor([[ 0.3816, -1.2462,  0.5695,  0.1170, -0.6605, -0.7185, -0.7190,  0.6389,
          0.7770, -0.1771,  0.1242,  0.4228,  0.0096, -0.2680,  0.1743, -0.2620,
         -1.3410, -0.1200,  0.2875, -0.1379],
        [ 0.4609, -1.3267,  0.6344,  0.2160, -0.6658, -0.7883, -0.7665,  0.7383,
          0.8392, -0.1539,  0.1131,  0.4253,  0.0094, -0.2943,  0.1922, -0.3628,
         -1.4549, -0.1028,  0.2432, -0.1585],
        [ 0.3823, -1.2470,  0.5702,  0.1180, -0.6605, -0.7192, -0.7194,  0.6399,
          0.7776, -0.1769,  0.1240,  0.4228,  0.0096, -0.2683,  0.1744, -0.2630,
         -1.3421, -0.1199,  0.2871, -0.1381],
        [ 0.3987, -1.2638,  0.5839,  0.1382, -0.6616, -0.7334, -0.7295,  0.6604,
          0.7898, -0.1731,  0.1221,  0.4239,  0.0084, -0.2746,  0.1778, -0.2839,
         -1.3654, -0.1160,  0.2776, -0.1412],
        [ 0.3659, -1.2301,  0.5562,  0.0977, -0.6595, -0.7049, -0.7093,  0.6193,
          0.7657, -0.1802,  0.1258,  0.4215,  0.0114, -0.2615,  0.

# A3C Code
- Simple A3C code

In [128]:
from torch import nn
import torch
import numpy as np


def v_wrap(np_array, dtype=np.float32):
    if np_array.dtype != dtype:
        np_array = np_array.astype(dtype)
    return torch.from_numpy(np_array)


def set_init(layers):
    for layer in layers:
        nn.init.normal_(layer.weight, mean=0., std=0.1)
        nn.init.constant_(layer.bias, 0.)


def push_and_pull(opt, lnet, gnet, done, s_, bs, buffer_action, buffer_reward, gamma):
    if done:
        v_s_ = 0.               # terminal
    else:
        v_s_ = lnet.forward(v_wrap(s_[None, :]))[-1].data.numpy()[0, 0]

    buffer_v_target = []
    for r in buffer_reward[::-1]:    # reverse buffer r
        v_s_ = r + gamma * v_s_
        buffer_v_target.append(v_s_)
    buffer_v_target.reverse()
    loss = lnet.loss_func(
        v_wrap(np.vstack(bs)),
        v_wrap(np.array(buffer_action), dtype=np.int64) if buffer_action[0].dtype == np.int64 else v_wrap(np.vstack(buffer_action)),
        v_wrap(np.array(buffer_v_target)[:, None]))

    # calculate local gradients and push local parameters to global
    opt.zero_grad()
    loss.backward()
    for lp, gp in zip(lnet.parameters(), gnet.parameters()):
        gp._grad = lp.grad
    opt.step()

    # pull global parameters
    lnet.load_state_dict(gnet.state_dict())


def record(global_ep, global_ep_r, ep_r, res_queue, name):
    with global_ep.get_lock():
        global_ep.value += 1
    with global_ep_r.get_lock():
        if global_ep_r.value == 0.:
            global_ep_r.value = ep_r
        else:
            global_ep_r.value = global_ep_r.value * 0.99 + ep_r * 0.01
    res_queue.put(global_ep_r.value)
    print(
        name,
        "Ep:", global_ep.value,
        "| Ep_r: %.0f" % global_ep_r.value,
    )

In [129]:
class SharedAdam(torch.optim.Adam):
    def __init__(self, params, lr=1e-3, betas=(0.9, 0.99), eps=1e-8,
                 weight_decay=0):
        super(SharedAdam, self).__init__(params, lr=lr, betas=betas, eps=eps, weight_decay=weight_decay)
        # State initialization
        for group in self.param_groups:
            for p in group['params']:
                state = self.state[p]
                state['step'] = 0
                state['exp_avg'] = torch.zeros_like(p.data)
                state['exp_avg_sq'] = torch.zeros_like(p.data)

                # share in memory
                state['exp_avg'].share_memory_()
                state['exp_avg_sq'].share_memory_()

In [130]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.multiprocessing as mp
import gym
import os
os.environ["OMP_NUM_THREADS"] = "1"

UPDATE_GLOBAL_ITER = 5
GAMMA = 0.9
MAX_EP = 3000

env = gym.make('CartPole-v0')
N_S = env.observation_space.shape[0]
N_A = env.action_space.n

print(N_S, N_A)

4 2


In [131]:
class Net(nn.Module):
    def __init__(self, s_dim, a_dim):
        super(Net, self).__init__()
        self.s_dim = s_dim
        self.a_dim = a_dim
        self.pi1 = nn.Linear(s_dim, 128)
        self.pi2 = nn.Linear(128, a_dim)
        self.v1 = nn.Linear(s_dim, 128)
        self.v2 = nn.Linear(128, 1)
        set_init([self.pi1, self.pi2, self.v1, self.v2])
        self.distribution = torch.distributions.Categorical

    def forward(self, x):
        pi1 = torch.tanh(self.pi1(x))
        logits = self.pi2(pi1)
        v1 = torch.tanh(self.v1(x))
        values = self.v2(v1)
        return logits, values

    def choose_action(self, s):
        self.eval()
        logits, _ = self.forward(s)
        prob = F.softmax(logits, dim=1).data
        m = self.distribution(prob)
        return m.sample().numpy()[0]

    def loss_func(self, s, a, v_t):
        self.train()
        logits, values = self.forward(s)
        td = v_t - values
        c_loss = td.pow(2)
        
        probs = F.softmax(logits, dim=1)
        m = self.distribution(probs)
        exp_v = m.log_prob(a) * td.detach().squeeze()
        a_loss = -exp_v
        total_loss = (c_loss + a_loss).mean()
        return total_loss

In [136]:
class Worker(mp.Process):
    def __init__(self, gnet, opt, global_ep, global_ep_r, res_queue, name):
        super(Worker, self).__init__()
        self.name = 'w%02i' % name
        self.g_ep, self.g_ep_r, self.res_queue = global_ep, global_ep_r, res_queue
        self.gnet, self.opt = gnet, opt
        self.lnet = Net(N_S, N_A)           # local network
        self.env = gym.make('CartPole-v0').unwrapped

    def run(self):
        total_step = 1
        while self.g_ep.value < MAX_EP:
            s = self.env.reset()
            buffer_s, buffer_action, buffer_reward = [], [], []
            ep_r = 0.
            while True:
#                 if self.name == 'w00':
#                     self.env.render()
                print(s)
                a = self.lnet.choose_action(v_wrap(s[None, :]))
                print(s[None, :])
                print(v_wrap(s[None, :]))
                s_, r, done, _ = self.env.step(a)
                if done: r = -1
                ep_r += r
                buffer_action.append(a)
                buffer_s.append(s)
                buffer_reward.append(r)

                if total_step % UPDATE_GLOBAL_ITER == 0 or done:  # update global and assign to local net
                    # sync
                    push_and_pull(self.opt, self.lnet, self.gnet, done, s_, buffer_s, buffer_action, buffer_reward, GAMMA)
                    buffer_s, buffer_action, buffer_reward = [], [], []

                    if done:  # done and print information
                        record(self.g_ep, self.g_ep_r, ep_r, self.res_queue, self.name)
                        break
                s = s_
                total_step += 1
        self.res_queue.put(None)

In [137]:
gnet = Net(N_S, N_A)        # global network
gnet.share_memory()         # share the global parameters in multiprocessing
opt = SharedAdam(gnet.parameters(), lr=1e-4, betas=(0.92, 0.999))      # global optimizer
global_ep, global_ep_r, res_queue = mp.Value('i', 0), mp.Value('d', 0.), mp.Queue()
print(mp.cpu_count())

# parallel training
workers = [Worker(gnet, opt, global_ep, global_ep_r, res_queue, i) for i in range(mp.cpu_count())]
[w.start() for w in workers]
res = []                    # record episode reward to plot
while True:
    r = res_queue.get()
    if r is not None:
        res.append(r)
    else:
        break
[w.join() for w in workers]

import matplotlib.pyplot as plt
plt.plot(res)
plt.ylabel('Moving average ep reward')
plt.xlabel('Step')
plt.show()

48
[-0.00884561  0.00451411  0.02020056 -0.04959281]
[[-0.00884561  0.00451411  0.02020056 -0.04959281]][-0.03547944  0.03127026 -0.03322896 -0.00991521]

tensor([[-0.0088,  0.0045,  0.0202, -0.0496]])[0.03297781 0.03484544 0.03839904 0.0296429 ]
[[-0.03547944  0.03127026 -0.03322896 -0.00991521]]

[-0.00875533 -0.19089158  0.01920871  0.2493945 ]
[[0.03297781 0.03484544 0.03839904 0.0296429 ]]tensor([[-0.0355,  0.0313, -0.0332, -0.0099]])[[-0.00875533 -0.19089158  0.01920871  0.2493945 ]]

[-0.04561013 -0.04901802  0.02548184 -0.01022353]
[-0.03485403 -0.16335977 -0.03342727  0.27210117]

tensor([[-0.0088, -0.1909,  0.0192,  0.2494]])tensor([[0.0330, 0.0348, 0.0384, 0.0296]])

[[-0.03485403 -0.16335977 -0.03342727  0.27210117]][-0.01257316 -0.38628251  0.0241966   0.54807383][[-0.04561013 -0.04901802  0.02548184 -0.01022353]][ 0.03367472 -0.16080554  0.0389919   0.33418957]



tensor([[-0.0349, -0.1634, -0.0334,  0.2721]])[[-0.01257316 -0.38628251  0.0241966   0.54807383]][[ 0.0336747

tensor([[-0.0494, -0.0107, -0.0496,  0.0243]])[[ 0.03490443 -0.36386443 -0.00310311  0.52437125]]tensor([[-0.0058, -0.1595, -0.0361,  0.2849]])[-0.00318032 -0.18588619  0.04444468  0.28906454]

[-0.01240207  0.00416165  0.03688031 -0.02538796]

[-0.04964606 -0.20511167 -0.04913658  0.30094164]tensor([[ 0.0349, -0.3639, -0.0031,  0.5244]])[-0.00901055 -0.35411626 -0.03035348  0.56602972]



[[-0.00318032 -0.18588619  0.04444468  0.28906454]]
[[-0.04964606 -0.20511167 -0.04913658  0.30094164]]tensor([[-0.0032, -0.1859,  0.0444,  0.2891]])[[-0.00901055 -0.35411626 -0.03035348  0.56602972]]


[-0.00689805 -0.38161282  0.05022597  0.59542698]tensor([[-0.0496, -0.2051, -0.0491,  0.3009]])
[[-0.01240207  0.00416165  0.03688031 -0.02538796]]
tensor([[-0.0090, -0.3541, -0.0304,  0.5660]])[[-0.00689805 -0.38161282  0.05022597  0.59542698]]


[-0.05374829 -0.3995001  -0.04311775  0.577732  ]tensor([[-0.0069, -0.3816,  0.0502,  0.5954]])

[-0.01609287 -0.15858193 -0.01903288  0.26394077][[-0.05374

[[-0.03259491 -0.19625529 -0.00646927  0.28604239]][ 2.65301958e-02 -4.27321016e-01  4.10024071e-04  5.35101328e-01][[-0.01438183  0.00532204  0.04834109  0.02043195]]
[-0.00828962 -0.24217029 -0.01328394  0.25464836]


tensor([[-0.0326, -0.1963, -0.0065,  0.2860]])[[ 2.65301958e-02 -4.27321016e-01  4.10024071e-04  5.35101328e-01]]tensor([[-0.0144,  0.0053,  0.0483,  0.0204]])[[-0.00828962 -0.24217029 -0.01328394  0.25464836]]



[-0.03652001 -0.39128438 -0.00074842  0.57667796][-0.01427539 -0.19045864  0.04874972  0.32796665]tensor([[ 2.6530e-02, -4.2732e-01,  4.1002e-04,  5.3510e-01]])tensor([[-0.0083, -0.2422, -0.0133,  0.2546]])[-0.03436576 -0.02172756  0.01512299 -0.02009141]




[[-0.03652001 -0.39128438 -0.00074842  0.57667796]][-0.01313303 -0.43710008 -0.00819097  0.54311188]

[[-0.01427539 -0.19045864  0.04874972  0.32796665]]
tensor([[-0.0365, -0.3913, -0.0007,  0.5767]])[[-0.01313303 -0.43710008 -0.00819097  0.54311188]]
[[-0.03436576 -0.02172756  0.01512299 -0.02009141]]
te

[[-0.01128557 -0.02748694  0.02480026  0.04939495]]tensor([[ 0.0179, -0.3617,  0.0327,  0.5992]])
[-0.00234936 -0.16427813  0.00970885  0.28763403][ 5.92540325e-04 -3.74311383e-01  7.26721970e-02  6.94861021e-01]


[ 0.01069548 -0.16702512  0.0446913   0.31698957]tensor([[-0.0113, -0.0275,  0.0248,  0.0494]])

[[ 5.92540325e-04 -3.74311383e-01  7.26721970e-02  6.94861021e-01]][[-0.00234936 -0.16427813  0.00970885  0.28763403]][-0.01183531 -0.22295557  0.02578816  0.34979825]
[[ 0.01069548 -0.16702512  0.0446913   0.31698957]]
[ 0.00679903 -0.01425005 -0.02309682 -0.03427595]

tensor([[-0.0023, -0.1643,  0.0097,  0.2876]])
tensor([[ 5.9254e-04, -3.7431e-01,  7.2672e-02,  6.9486e-01]])
[[-0.01183531 -0.22295557  0.02578816  0.34979825]]
tensor([[ 0.0107, -0.1670,  0.0447,  0.3170]])
[-0.00563492 -0.35953719  0.01546153  0.58336316]
tensor([[-0.0118, -0.2230,  0.0258,  0.3498]])
[ 0.00735498 -0.36275419  0.05103109  0.62342489]
[[ 0.00679903 -0.01425005 -0.02309682 -0.03427595]]
[-0.01629

tensor([[ 0.0310,  0.0415, -0.0307,  0.0159]])tensor([[ 0.0031, -0.3791, -0.0071,  0.5392]])
tensor([[ 0.0260, -0.1981, -0.0264,  0.2947]])

[ 0.0318634  -0.15315385 -0.03036728  0.2987702 ][-0.00446788 -0.18390651  0.0036747   0.2442871 ][ 0.02204844 -0.392806   -0.02054237  0.57891439]


[[ 0.0318634  -0.15315385 -0.03036728  0.2987702 ]][[-0.00446788 -0.18390651  0.0036747   0.2442871 ]][[ 0.02204844 -0.392806   -0.02054237  0.57891439]][ 0.02492307 -0.0487669  -0.04054778  0.03129896]



tensor([[-0.0045, -0.1839,  0.0037,  0.2443]])tensor([[ 0.0319, -0.1532, -0.0304,  0.2988]])tensor([[ 0.0220, -0.3928, -0.0205,  0.5789]])


[-0.00814601 -0.37908075  0.00856044  0.53812685][ 0.02880033 -0.34783007 -0.02439188  0.58172329]
[[ 0.02492307 -0.0487669  -0.04054778  0.03129896]]

[[-0.00814601 -0.37908075  0.00856044  0.53812685]]
[[ 0.02880033 -0.34783007 -0.02439188  0.58172329]]
tensor([[ 0.0249, -0.0488, -0.0405,  0.0313]])tensor([[-0.0081, -0.3791,  0.0086,  0.5381]])

tensor([[ 0.

KeyboardInterrupt: 